In [1]:
%load_ext autotime

In [2]:
%load_ext autoreload
%autoreload 2

PREDICTION_PATH = '../data/predictions/'

time: 331 ms


#### Load data

In [3]:
from datafile_methods.data_io import load_datasets, load_datasets_sur

# Load datasets
folds, ratings, sample_submission = load_datasets()

folds_tr, folds_te, ratings_sur, sample_submission_sur = load_datasets_sur()

time: 1min 26s


#### This should be moved to a different file!
Function to carry out the cross-validation

In [6]:
import numpy as np
from functools import reduce

def cross_validation(folds, prediction_model, args):
    """Gets the training and test errors for all the folds.
    Returns two lists including train and test errors respectively
    """
    train_rmse = np.zeros(len(folds))
    test_rmse = np.zeros(len(folds))
    for i in range(len(folds)):
        folds_copy = folds.copy()
        test = folds_copy.pop(i)
        train = reduce(lambda x, y: x + y, [m for m in folds_copy])
        train_rmse[i], test_rmse[i] = prediction_model(train, test,
            True, fold_number=i, **args)
    return train_rmse, test_rmse

time: 27.4 ms


In [12]:
def cross_validation_sur(folds_tr, folds_te, prediction_model, args):
    """Gets the training and test errors for all the folds.
    Returns two lists including train and test errors respectively
    """
    train_rmse = np.zeros(len(folds_tr))
    test_rmse = np.zeros(len(folds_tr))
    for i in range(len(folds_tr)):
        train = folds_tr[i]
        test = folds_te[i]
        train_rmse[i], test_rmse[i] = prediction_model(train, test,
            True, fold_number=i, **args)
    return train_rmse, test_rmse

time: 25.6 ms


#### Define dictionaries to keep the train and test errors

In [8]:
errors = {}

time: 11.7 ms


#### Train models

Baselines

In [9]:
from prediction_methods.baseline_model import model_baseline

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_baseline, {'prediction_path': PREDICTION_PATH})
errors['baseline'] = [train_rmse, test_rmse]

# Train on entire training set. Predict for `submission` dataset
model_baseline(ratings, sample_submission, False, prediction_path=PREDICTION_PATH)

time: 58.2 s


Matrix factorization with ALS

In [ ]:
from prediction_methods.mf_als_model import model_mf_als

u_feats = np.loadtxt("u_feats.txt")
i_feats = np.loadtxt("i_feats.txt")

args = {'prediction_path': PREDICTION_PATH,
        'k': 20,
        'lambda_u': .1,
        'lambda_i': .1,
        'tol': 1e-4,
        'max_iter': 100,
        'init_u_features': u_feats,
        'init_i_features': i_feats}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_mf_als, args)
errors['mf_als'] = [train_rmse, test_rmse]

# Train on entire training set. Predict for `submission` dataset
model_mf_als(ratings, sample_submission, False, **args)

## 1h 37min 47s

Matrix factorization with SVD (scipy)

In [10]:
from prediction_methods.mf_svd_model import model_mf_svd

args = {'prediction_path': PREDICTION_PATH,
        'k': 13,
        'library': 'scipy',
        'fn_suffix': 'sci_'}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_mf_svd, args)
errors['mf_svd'] = [train_rmse, test_rmse]

# Train on entire training set. Predict for `submission` dataset
model_mf_svd(ratings, sample_submission, False, **args)

## Before ref. achieved 0.99385 on test. It's better to average it with the next ones!

time: 1min 27s


Matrix factorization with SVD (sklearn)

In [11]:
args = {'prediction_path': PREDICTION_PATH,
        'k': 13,
        'n_iter': 50,
        'library': 'sklearn',
        'random_state': 70,
        'fn_suffix': 'skl_'}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_mf_svd, args)
errors['mf_svd_skl'] = [train_rmse, test_rmse]

# Train on entire training set. Predict for `submission` dataset
model_mf_svd(ratings, sample_submission, False, **args)

### Bef ref. achieved 0.99386 (approx) on test
# Achieves 0.99082 in Kaggle

time: 1min 40s


Recommend's ALS model

In [ ]:
from prediction_methods.recommend_model import model_mf_als_recommend

n_item, n_user = ratings.shape
args = {'n_user': n_user,
        'n_item': n_item,
        'prediction_path': PREDICTION_PATH,
        'k': 20,
        'n_iter': 50,
        'reg': 0.085}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_mf_als_recommend, args)
errors['mf_als_recommend'] = [train_rmse, test_rmse]
# time 20 min approx

# Train on entire training set. Predict for `submission` dataset
model_mf_als_recommend(ratings, sample_submission, False, **args)

#

Surprise models

* SlopeOne

In [14]:
from prediction_methods.surprise_models import model_slope_one

args = {'prediction_path': PREDICTION_PATH}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation_sur(folds_tr, folds_te, model_slope_one, args)
errors['slope_one'] = [train_rmse, test_rmse]

model_slope_one(ratings_sur, sample_submission_sur, False, **args)

time: 4h 53min 9s


* KNN Baseline (user based) - Verbose

In [15]:
from prediction_methods.surprise_models import model_knn_baseline

args = {'prediction_path': PREDICTION_PATH,
        'k': 300,
        'min_k': 20,
        'name': 'pearson_baseline',
        'user_based': True,from prediction_methods.mf_als_model import model_mf_als

args = {'prediction_path': PREDICTION_PATH,
        'k': 20,
        'lambda_u': .1,
        'lambda_i': .1,
        'tol': 1e-4,
        'max_iter': 100,
        'init_u_features': u_feats,
        'init_i_features': i_feats}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation(folds, model_mf_als, args)
errors['mf_als'] = [train_rmse, test_rmse]

# Train on entire training set. Predict for `submission` dataset
model_mf_als(ratings, sample_submission, False, **args)

## Before refactoring achieved 0.9878 in training
## Other good param_combs are (k20, lambdas 0.095) and (k30, lambdas 0.095)
        'fn_suffix': 'u_'}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation_sur(folds_tr, folds_te, model_knn_baseline, args)
errors['knn_baseline_u'] = [train_rmse, test_rmse]
# time: 20min 56s

model_knn_baseline(ratings_sur, sample_submission_sur, False, **args)
# time: 10min 7s

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
time: 1h 25min 44s


* KNN Baseline (item based) - Verbose

In [16]:
args = {'prediction_path': PREDICTION_PATH,
        'k': 60,
        'min_k': 20,
        'name': 'pearson_baseline',
        'user_based': False,
        'fn_suffix': 'i_'}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation_sur(folds_tr, folds_te, model_knn_baseline, args)
errors['knn_baseline_i'] = [train_rmse, test_rmse]
# time: 21min 22s

model_knn_baseline(ratings_sur, sample_submission_sur, False, **args)
# time: 9min 49s

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
time: 1h 10min 57s


* NMF

In [17]:
from prediction_methods.surprise_models import model_nmf

args = {'prediction_path': PREDICTION_PATH,
        'biased': True,
        'k': 22,
        'reg_pu': 0.05,
        'reg_qi': 0.05,
        'reg_bu': 0.055,
        'reg_bi': 0.055,
        'n_epochs': 150}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation_sur(folds_tr, folds_te, model_nmf, args)
errors['nmf'] = [train_rmse, test_rmse]
# time: 12min 57s

model_nmf(ratings_sur, sample_submission_sur, False, **args)
# time: 3min 24s

time: 32min 21s


* SVD

In [18]:
from prediction_methods.surprise_models import model_svd

args = {'prediction_path': PREDICTION_PATH,
        'biased': True,
        'k': 130,
        'reg_all': 0.08,
        'n_epochs': 50}

# 5-fold cross-validation
train_rmse, test_rmse = cross_validation_sur(folds_tr, folds_te, model_svd, args)
errors['svd'] = [train_rmse, test_rmse]
# time: 11min 16s

model_svd(ratings_sur, sample_submission_sur, False, **args)
# time: 3min 2s

time: 28min 5s


In [19]:
errors

{'baseline': [array([ 0.99425462,  0.99468217,  0.99438628,  0.99377059,  0.99424295]),
  array([ 1.00601127,  1.00459408,  1.00499478,  1.00810896,  1.00620091])],
 'knn_baseline_i': [array([ 0.91587251,  0.91626753,  0.91588456,  0.9155493 ,  0.9158178 ]),
  array([ 0.99486469,  0.99258583,  0.99411646,  0.99657762,  0.99476892])],
 'knn_baseline_u': [array([ 0.95829473,  0.95866049,  0.95840947,  0.95780609,  0.95822906]),
  array([ 0.99762091,  0.99615513,  0.99711687,  0.99980506,  0.99795458])],
 'mf_svd': [array([ 0.95135504,  0.9518707 ,  0.95149805,  0.9507868 ,  0.95116691]),
  array([ 0.99601752,  0.99452261,  0.99532743,  0.9982914 ,  0.9965627 ])],
 'mf_svd_skl': [array([ 0.95135528,  0.95187106,  0.95149846,  0.95078682,  0.95116699]),
  array([ 0.99601761,  0.99452242,  0.99532679,  0.99829138,  0.996563  ])],
 'nmf': [array([ 0.95691839,  0.95766328,  0.95700982,  0.95695702,  0.95728115]),
  array([ 0.99564831,  0.99380356,  0.99527643,  0.99698293,  0.99576552])],
 's

time: 236 ms


#### Combine ratings

Load ratings for test and validation files

In [ ]:
#TODO

Load ratings for submission files

In [ ]:
#TODO

Define ensembling method

In [ ]:
#TODO

Apply previous weights to obtain a final submission

In [ ]:
#TODO